# Creating InstructLab Taxonomies

<ul>
<li>Contributors: InstructLab team and IBM Research Technology Education team:
<li>Questions and support: kochel@us.ibm.com, IBM.Research.JupyterLab@ibm.com
<li>Release date: 2025-05-06
</ul>

# Overview
This Jupyter notebook facilitates compiling taxonomies for InstructLab, an open source AI project that enables knowledge and skills contributions to Large Language Models (LLMs). This notebook performs the following:
1. Accepts one or more of Question and Answer (QNA) files as input
1. Performs yamllint checks on the QNA files to verify their formant
1. Places the QNA files in the desired palce in a taxonomy
1. Verifies the taxonomy by running the ilab diff function
1. Creates a Tar file of the taxonomy and provides it to the Red Hat AI Service for Synthetic Data Generation

This notebook can be run with the free Colab environment.


# Step 1. Clone the Instructlab Environment and Present Run Options

Replicate the ilab data repository containing the pip requirements and data files.


After selecting parameters, the remainder of this notebook can be run either:
- Running All Cells by selecting Runtime->Run cell and below
- Cell by cell by selecting the arrow on each code cell and running them sequentially.

Run this next cell, select the following parameters, then follow the direction in the following cell to run the rest of this notebook.

We've provided Question and Answer files for these datasets: "2024 Oscar Awards Ceremony", "Quantum Roadmap and Patterns" and "Artificial Intelligence Agents".

The "Multi-QNA Example" dataset contains QNA files for Oscars, Quantum and Agentic AI to show now multiple QNA files can be provided and processed.

Feel free to choose one of these datasets, or select "Your Content 1" or "Your Content 2" and follow the instructions below to provide your own data.

In [2]:
# Install these items first to avoid a later reset
!pip install psutil==7.0.0 pillow==10.4.0 --quiet

import os
os.chdir('/content/')
if os.path.exists("ilab"):
    !rm -rf ilab
!git clone https://github.com/KenOcheltree/ilab-test.git --quiet --recurse-submodules ilab

# Run this second cell to show parameters
import ipywidgets as widgets
data_set = widgets.ToggleButtons(
    options=['2024 Oscars', 'Quantum', 'Agentic AI', 'Multi-QNA Example', 'Your Content 1', 'Your Content 2'],
    description='Dataset:', style={"button_width": "auto"}
)
print("\nSelect the Dataset for this run:")
display(data_set)
print("After selecting the dataset, select the next cell and then choose Runtime->Run cell and below")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.5 MB/s eta 0:00:00

Select the Dataset for this run:


ToggleButtons(description='Dataset:', options=('2024 Oscars', 'Quantum', 'Agentic AI', 'Multi-QNA Example', 'Y…

After selecting the dataset, select the next cell and then choose Runtime->Run cell and below


#Step 2. Prepare for Creating the Taxonomy

## 2.1 Place your IBM Cloud and COS Access Credentials in Secrets

When you configured IBM Cloud and COS access, you will have access keys and resource IDs that you need to provide to upload your taxonomy to the Red Hat AI Service. You need to set the following paramters in your secrets area (accessed by selecting the key icon on the left). Be sure to give notebook access to each of these parameters:

- ibmcloud_key - a key to access your IBM Cloud account of the form "XX_XXXXXXXXXXXXXXXXXX"
- ibmcloud_region - a value specifying your IBM Cloud region of the form "us-east"
- ibmcloud_resource - a designator for yout resource of the form "InstructLab"
- cos_id - an ID for your COS of the form "crn:v1:bluemix:public:cloud-object-storage:global:a/0450...f04c::"
- cos_api_key - an API key for accessing your COS of the form "XXX...XXX"
- cos_bucket=userdata.get("cos_bucket") - the name of your COS bucket of the form "ilabdata"

## 2.2 Provide the Taxonomy data

You may want to run this notebook once with an existing dataset before creating your own to understand the Tasonomy creation flow.

You can optionally provide your own InstructLab QNA file for processing in this step. Follow these steps to add your own dataset:
1. Create your own qna.yaml file following the directions on the InstructLab taxonomy [readme](https://github.com/instructlab/taxonomy).
1. After creating your qna.yaml file, add a comment in the first line that starts with *#Location:* and specifies the location of the file in the taxonomy. For example, a quantum computing qna.yaml file would have the following for the first line:
```
#location: /knowledge/information/computer_science/quantum_computing'
```
1. Add your qna.yaml to the /content/ilab/data/your_content_1 folder or the /content/ilab/data/your_content_2 folder by dragging and dropping them in the desired folder.
1. If you want to include multiple qna.yaml files in your taxonomy, add a unique identifer "NNN" to the name so it is of the form qnaNNN.yaml. Any number of QNA files can be included as long as they have unique names
1. You can now specify to run with your own data by selecting **Your Content 1** or **Your Content 2** in the next code cell.

##2.3 Complete the Environment Set Up


This code cell installs the remainder of the reuired pip packages and takes a few minutes to run.

The InstructLab configuration is captured in the *config.yaml* file. This step creates the config.yaml file and sets **taxomony_path = taxonomy** - the root location of the taxonomy is set to the taxonomy folder in instructlab-latest

**Note:** Ignore any pip inconsistency errors or warnings in the installation. They are inconsequential to the running of this notebook.

**Note** If you perform **Runtime->Run cell and below** on this cell, the rest of notebook will take about 10 minutes to run. After running, it will present a prompt for providing questions to the pre-trained and trained models to test improvements in the model.

In [3]:
# You can run the rest of the notebook by selecting this cell and choosing "Runtime->Run cell and below"

# Acquire access to secret keys
from google.colab import userdata

# Wrap Code cell output
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Install the ibmcloud plugin
!curl -fsSL https://clis.cloud.ibm.com/install/linux | sh
!ibmcloud plugin install ilab -f

# Install the rest of the requirements
os.chdir('/content/ilab/')
print("Starting directory: "+ os.getcwd())
!pip install -r requirements.txt --quiet
!ilab system info

from IPython.display import Image, display
import shutil

# Initialize ilab
base_dir="/root/"
taxonomy_path='taxonomy'
model_path = "models/granite-7b-lab-Q4_K_M.gguf"

# Remove old ilab configuration data
if os.path.exists(base_dir+".local/share/instructlab"):
    print("removing " + base_dir+".local/share/instructlab")
    shutil.rmtree(base_dir+".local/share/instructlab")

# Initialize local instructlab isntall
print("Initialized ilab")
command = f"""
ilab config init<<EOF
{taxonomy_path}
Y
{model_path}
0
EOF
"""
# Using the ! operator to run the command
!echo "Running ilab config init"
!{command}

Current platform is linux64. Downloading corresponding IBM Cloud CLI...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.2M  100 10.2M    0     0  76.6M      0 --:--:-- --:--:-- --:--:-- 76.8M
Download complete. Executing installer...
Bluemix_CLI/
Bluemix_CLI/install_bluemix_cli
Bluemix_CLI/autocomplete/
Bluemix_CLI/autocomplete/zsh_autocomplete
Bluemix_CLI/autocomplete/bash_autocomplete
Bluemix_CLI/bin/
Bluemix_CLI/bin/ibmcloud
Bluemix_CLI/bin/LICENSE
Bluemix_CLI/bin/ibmcloud.sig
Bluemix_CLI/bin/NOTICE
Bluemix_CLI/install
Bluemix_CLI/uninstall
Install complete.
Looking up 'ilab' from repository 'IBM Cloud'...
Plug-in 'ilab 0.0.22' found in repository 'IBM Cloud'
Attempting to download the binary file...
 20.24 MiB / 20.24 MiB  100.00% 0s
21221216 bytes downloaded
Installing binary...
OK
Plug-in 'ilab 0.0.22' was successfully installed into /root/.bluemix/plugins/ilab. Us

# Step 3. Initialize Red Hat AI InstructLab Access

In [5]:
print("Initialize Red Hat AI InstructLab Access")
print("Installing the IBMCloud ilab plugin")

import os
import subprocess
import time
import ibm_boto3
from ibm_botocore.client import Config
from ibm_botocore.exceptions import ClientError

# Pull data from secrets
ibmcloud_key=userdata.get("ibmcloud_key")
ibmcloud_region=userdata.get("ibmcloud_region")
ibmcloud_resource=userdata.get("ibmcloud_resource")
cos_bucket=userdata.get("cos_bucket")

!ibmcloud config --check-version=false
shell_command = f"ibmcloud login -apikey {ibmcloud_key} -r {ibmcloud_region} -g {ibmcloud_resource}"
!{shell_command}

# !ibmcloud resource service-instances --service-name instructlab --long
proj_index=0
response = subprocess.check_output("ibmcloud resource service-instances --service-name instructlab --long", shell=True).decode("utf-8").split()
for index, word in enumerate(response):
    if word == "GUID:":
        proj_index=index+1
        break
if proj_index==0:
    print("Assign project-id")
    !ibmcloud resource service-instance-create 'instructlab' instructlab instructlab-pricing-plan us-east
    for index, word in enumerate(response):
        if word == "GUID:":
            proj_index=index+1
            break
if proj_index==0:
    print("ERROR in assigning Project ID")
project_id=response[proj_index]

shell_command = f"ibmcloud ilab config set project-id {project_id}"
!{shell_command}

print("Check IBM Cloud COS authorization policies")
!ibmcloud iam authorization-policies

# Set up COS Access Here
print("Set up COS storage and check access")
endpoint_url = f'https://s3.{ibmcloud_region}.cloud-object-storage.appdomain.cloud'
# Current list of auth_endpoints is at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'


Initialize Red Hat AI InstructLab Access
Installing the IBMCloud ilab plugin
API endpoint: https://cloud.ibm.com
Authenticating...
OK

Targeted account IBM - Tyler's Account (837bf8218846470cad3ecf8bf7dcc43b) <-> 2095376

Targeted resource group default

Targeted region us-east


API endpoint:     https://cloud.ibm.com
Region:           us-east
User:             krglosse@us.ibm.com
Account:          IBM - Tyler's Account (837bf8218846470cad3ecf8bf7dcc43b) <-> 2095376
Resource group:   default
OK
Check IBM Cloud COS authorization policies
Getting authorization policies under account 837bf8218846470cad3ecf8bf7dcc43b as krglosse@us.ibm.com...
OK

ID:                        1e8d215c-6e09-4d45-ad04-d4c0650dde0a
Source service account:    8d28df3cd15b4072a69a252c0b2188fc
Source service name:       instructlab
Source service instance:   6a0f84d6-d9a9-40e2-882b-8599a8b07778
Target service name:       secrets-manager
Target service instance:   ad83723e-8cbd-400d-9ec8-c8f35ab65125
Roles:        


# Step 4. Check the Format of the QNA YAML Files

Running this cell checks the format of the yaml files before they are placed in the taxonomy to ensure they are the right length and there are no trailing blanks.

**Important:** Rerun the following cell until all of the QNA files pass the yamllint test. Otherwise the file will fail in the Synthetic Data Generation step.


In [8]:
import yamllint
# Select the folder of the dataset
use_cases = {"2024 Oscars": "oscars", "Quantum": "quantum", "Agentic AI": "agentic_ai",
            "Multi-QNA Example": "example","Your Content 1": "your_content_1", "Your Content 2": "your_content_2"}
use_case = use_cases[data_set.value]
qna_dir = "data/" + use_case + "/"
print("Running yaml checker on " + data_set.value + " data in folder " + qna_dir)
for f in os.listdir(qna_dir):
    f=f.lower()
    if f.startswith('qna'):
      print("Checking File: " + f)
      yaml_file = qna_dir + f
      shell_command = f"yamllint /content/ilab/{yaml_file} -c /content/ilab/yamlrules.yaml"
      !{shell_command}

Running yaml checker on Your Content 1 data in folder data/your_content_1/
Checking File: qna.yaml


# Step 5. Create the Taxonomy with the QNA Files
Running this next cell places the QNA files in the proper directories of the taxonomy.

If you want to add additional QNA files to the taxonomy after the following cell is run, you can create the necessary levels of directories and add the qna.yaml named file directly to the taxonomy.


In [11]:
# List all of the files in the use_case directory that begin with QNA
print_lines=30
for f in os.listdir(qna_dir):
    f=f.lower()
    if f.startswith('qna'):
        qna_file = qna_dir + f
        print("Show the QNA file: " + qna_file)
        with open(qna_file, 'r') as input_file:
            for line_number, line in enumerate(input_file):
                if line_number == 0:
                    words = line.split()
                    print(words)
                    print(words[0])
                    if words[0] == "# location:" and len(words) == 2:
                      qna_location = words[1]
                    else:
                      print("ERROR: No specificed location found in QNA File: " + qna_file)
                      break
                if line_number > print_lines:  # line_number starts at 0.
                    break
                print(line_number, line, end="")
        # Place the QNA file in the proper taxonomy directory if it does not already exist
        new_qna_dir = "/taxonomy" + qna_location
        if os.path.exists(os.getcwd()+new_qna_dir):
            print("\nWARNING: QNA file already exists in the taxonomy at duplicate location, not inserting")
        else:
            print("\nPlace QNA file in taxononmy as: /taxonomy"+qna_location+"/qna.yaml")
            shell_command1 = f"mkdir -p ./taxonomy{qna_location}"
            shell_command2 = f"cp ./{qna_file} ./taxonomy{qna_location}/qna.yaml"
            !{shell_command1}
            !{shell_command2}

Show the QNA file: data/your_content_1/qna.yaml
['#', 'location:', '/knowledge/astrology/test/phoenix']
#
ERROR: No specificed location found in QNA File: data/your_content_1/qna.yaml


NameError: name 'qna_location' is not defined

# Step 6. Verify the Taxonomy Data Repository
Run diff to verify the taxonomy. Record the errors on this step and correct them in your QNA files and then rerun the notebook with the corrected QNA files.

In [ ]:
print("Verify the taxonomy")
!ilab taxonomy diff

# Step 7. Add the Taxonomy to the Cloud


In [ ]:
set_name=use_case
tax_dir= os.getcwd()+"/taxonomy"
shell_command = f"ibmcloud ilab taxonomy add --name {set_name} --taxonomy-path {tax_dir} \
--cos-endpoint https://s3.us-east.cloud-object-storage.appdomain.cloud \
--cos-id {cos_id} \
--cos-bucket {cos_bucket}"

print("Add the taxonomy to the cloud")
tax_response = subprocess.check_output(shell_command, shell=True)
print("Taxonomy added")

response= tax_response.decode("utf-8").split()
for index, word in enumerate(response):
    if word == "id":
        break
tax_id = response[index+1]

print("taxonomy id = " + tax_id)

<a id="IL3_learn"></a>
# Learn More

InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081).

This notebook is based on the InstructLab CLI repository available [here](https://github.com/instructlab/instructlab).

Contact us by email to ask questions, discuss potential use cases, or schedule a technical deep dive. The contact email is IBM.Research.JupyterLab@ibm.com.

© 2025 IBM Corporation